In [2]:
#pip install mesa

     |████████████████████████████████| 1.8 MB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 241 kB 27.3 MB/s eta 0:00:01
     |████████████████████████████████| 189 kB 25.5 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 17.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 13.1 MB/s eta 0:00:01
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.12.0
    Uninstalling Pygments-2.12.0:
      Successfully uninstalled Pygments-2.12.0
Note: you may need to restart the kernel to use updated packages.


In [26]:

from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
import random
import numpy as np
import pandas as pd
import networkx as nx
import scipy.stats

## The Agents

In [27]:
class TRScientist(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.record = [] # Track record
        self.m = round(random.uniform(0.05, 0.5), 2) # Open-mindedness
        self.model = model
        self.unique_id = unique_id
        self.hyp = np.round(np.arange(0, 1.001, 1/5), 2)
        self.cred = np.round(np.full(len(self.hyp), 1/len(self.hyp)), 2) # Credence for each hyp
        self.noise = random.uniform(0.001, 0.2) # equivalent to sigma in paper
        self.c = round(random.random(), 2) # weight for evidence vs testimony
        self.neighbors = [] # trusted informants
        self.social = None 
        self.evidential = None
        self.pr = 0
        self.ev = 0
        self.id = 0
        self.hub = 0
        self.authority = 0
        self.Brier = [] # Prediction at previous time step against new toss
        self.BrierT = None # Cred against truth (God's eye view)
        self.crps = None
    def __hash__(self):
        return hash((self.model, self.unique_id))
    def __eq__(self, other):
        return (self.model, self.unique_id) == (other.model, other.unique_id)
    def r_avg(self):
        if len(self.record) > 0:
            # Mean Brier so far for toss predictions
            return round(sum(self.record)/len(self.record), 4)
        else:
            return 1
    def update_social(self):
        self.social = np.round(sum([a.cred for a in self.neighbors])/len(self.neighbors), 4)
    def update_evidence(self):
        toss = np.random.binomial(1, self.model.truth)
        self.Brier.append(round((toss - sum(self.cred*self.hyp))**2, 4)) # Cred at previous time step against new toss
        Pr_E_H = np.absolute((1-toss)-self.hyp)
        posterior = Pr_E_H*self.cred/np.sum(self.cred*Pr_E_H)
        loc = posterior
        scale = self.noise
        # No neg credence
        noisy = scipy.stats.truncnorm.rvs((0.0001-loc)/scale, (9.9999-loc)/scale, loc=loc, scale=scale)
        # Normalize
        self.evidential = noisy/sum(noisy)
    def update_neighbors(self):
        n = round(len(self.model.schedule.agents)*self.m)
        if n < 1:
            # Agent trust no one
            self.neighbors = [self]
        elif len(self.record) == 0:
            # No track records yet
            self.neighbors = random.sample(self.model.schedule.agents, n)
        else:
            # Choose best performing agents so far
            temp = []
            ls = self.model.schedule.agents
            random.shuffle(ls)
            temp = sorted(ls, key=lambda x: x.r_avg())[:n]
            if len(temp) < 1:
                temp.append(self)
            self.neighbors = temp
    def step(self):
        self.update_evidence()
        self.update_neighbors()
        self.update_social()
    def advance(self):
        # linear combination of social and evidential components
        new_cred = np.round((1-self.c)*self.social + self.c*self.evidential, 2) 
        self.cred = new_cred
        # calculate inaccuracy
        t = np.zeros((len(self.hyp),)) 
        t[int(self.model.truth*5)] = 1 # array of truth value for each hypothesis
        self.BrierT = round(sum((self.cred-t)**2), 4)
        self.crps = crps(self.cred, self.model.truth)

In [28]:
class RandomScientist(TRScientist):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
    def update_neighbors(self):
        n = round(len(self.model.schedule.agents)*self.m)
        if n < 1:
            self.neighbors = [self]
        else:
            self.neighbors = random.sample(self.model.schedule.agents, n)

In [29]:
class PatientScientist(TRScientist):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.c = 1

In [30]:
def Euclidean(x, y):
    return sum((x-y)**2)

In [31]:
def crps(cred, truth):
    penalty = 0
    for i in range(len(cred)):
        if i<(truth*5):
            penalty += (sum(cred[:i+1])-0)**2
        else:
            penalty += (sum(cred[:i+1])-1)**2
    return(round(penalty, 4))

## The Model

In [19]:
class TRModel(Model):
    def __init__(self, truth, feedback_rate):
        self.truth = truth
        self.schedule = SimultaneousActivation(self)
        self.fbr = feedback_rate # How often are track records public? Always 1 in current model.
        self.schedule = SimultaneousActivation(self)
        for i in range(30):
            self.schedule.add(TRScientist(i, self))
        self.datacollector = DataCollector(
            agent_reporters={"pr": "pr", "ev": "ev",
                             "id": "id", "hub": "hub", "authority": "authority",
                             "Brier": "Brier", "BrierT": "BrierT", "crps": "crps",
                             "neighbors": get_neighbors, 
                             "c": "c", "m": "m", "noise": "noise",
                             "cred": "cred", "social": "social", "evidential": "evidential"},
            model_reporters={"truth": "truth"})
    def centrality(self):
        G = nx.DiGraph()
        for a in self.schedule.agents:
            G.add_node(a)
            for n in a.neighbors:
                if a != n:
                    G.add_edge(a, n)
        idc = nx.in_degree_centrality(G)
        evc = nx.eigenvector_centrality_numpy(G)
        pr = nx.pagerank(G)
        hub, authority = nx.hits(G)
        for a in self.schedule.agents:
            a.pr = round(pr[a], 4)
            a.ev = round(evc[a], 4)
            a.id = round(idc[a], 4)
            a.hub = round(hub[a], 4)
            a.authority = round(authority[a], 4)
    def step(self):
        self.schedule.step()
        if np.random.binomial(1, self.fbr):
            for a in self.schedule.agents:
                a.record.append(a.Brier[-1])
        self.centrality()
        self.datacollector.collect(self)

In [32]:
class RandomModel(TRModel):
    def __init__(self, truth, feedback_rate):
        super().__init__(truth, feedback_rate)
        self.schedule = SimultaneousActivation(self)
        for i in range(15):
            self.schedule.add(TRScientist(i, self))
        for i in range(15, 30):
            self.schedule.add(RandomScientist(i, self))
        self.datacollector = DataCollector(
            agent_reporters={"pr": "pr", "ev": "ev", 
                             "id": "id", "hub": "hub", "authority": "authority",
                             "Brier": "Brier", "BrierT": "BrierT", "crps": "crps",
                             "neighbors": get_neighbors, 
                             "c": "c", "m": "m", "noise": "noise",
                             "cred": "cred", "social": "social", "evidential": "evidential"},
            model_reporters={"truth": "truth"})

In [33]:
class PatientModel(TRModel):
    def __init__(self, truth, feedback_rate):
        super().__init__(truth, feedback_rate)
        self.schedule = SimultaneousActivation(self)
        for i in range(30):
            self.schedule.add(PatientScientist(i, self))
        self.datacollector = DataCollector(
            agent_reporters={"pr": "pr", "ev": "ev", 
                             "id": "id", "hub": "hub", "authority": "authority",
                             "Brier": "Brier", "BrierT": "BrierT", "crps": "crps",
                             "neighbors": get_neighbors, 
                             "c": "c", "m": "m", "noise": "noise",
                             "cred": "cred", "social": "social", "evidential": "evidential"},
            model_reporters={"truth": "truth"})
    def activate(self):
        for a in self.schedule.agents:
            a.c = round(random.random(), 2)

In [34]:
def get_neighbors(agent):
    return [a.unique_id for a in agent.neighbors]

## Run Simulation

#### Baseline, original, less-monopoly

In [35]:
main = pd.DataFrame()

for run in range(1):
    t = random.choice(np.round(np.arange(0, 1.001, 1/5), 2))
    f = 1
    m = RandomModel(t, f)
    for i in range(100):
        m.step()
    df = m.datacollector.get_agent_vars_dataframe()
    df['truth'] = t
    df['f_rate'] = f
    df['run'] = run
    main = pd.concat([main, df])

#### More-patience

In [36]:
main = pd.DataFrame()

for run in range(1):
    t = random.choice(np.round(np.arange(0, 1.001, 1/5), 2))
    f = 1
    m = PatientModel(t, f)
    for i in range(50):
        m.step()
    # Begin assessing peers after 50 time steps
    m.activate()
    for i in range(50):
        m.step()
    df = m.datacollector.get_agent_vars_dataframe()
    df['truth'] = t
    df['f_rate'] = f
    df['run'] = run
    main = pd.concat([main, df])

In [37]:
main.dropna(inplace=True)
main.to_csv('test.csv')